<a href="https://colab.research.google.com/github/kwlaial/Music_Sentiment_Detection_to_Images/blob/main/Music_Sentiment_Detection_to_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Music Sentiment Detection to Images

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d cakiki/muse-the-musical-sentiment-dataset

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.9/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.9/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense, Embedding, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

In [ ]:
# Load the dataset
df = pd.read_csv("/content/muse_v3.csv")
print(df["seeds"])


0                                    ['aggressive']
1                                    ['aggressive']
2                                    ['aggressive']
3        ['aggressive', 'fun', 'sexy', 'energetic']
4                                    ['aggressive']
                            ...                    
89996                               ['transparent']
89997                               ['transparent']
89998                               ['transparent']
89999                               ['transparent']
90000                               ['translucent']
Name: seeds, Length: 90001, dtype: object


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize a LabelEncoder object
label_encoder = LabelEncoder()

# Fit the encoder to the target variable
label_encoder.fit(train_df['seeds'])

# Transform the target variable to a numeric format
train_y = label_encoder.transform(train_df['seeds'])
test_y = label_encoder.transform(test_df['seeds'])

ValueError: ignored

In [ ]:
# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Print the size of each set
print("Train set size:", len(train_df))
print("Test set size:", len(test_df))

Train set size: 72000
Test set size: 18001


In [ ]:
# Tokenize the text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_df['seeds'])
train_sequences = tokenizer.texts_to_sequences(train_df['seeds'])
test_sequences = tokenizer.texts_to_sequences(test_df['seeds'])

In [ ]:
# Pad the sequences to a fixed length
max_length = 100
train_data = sequence.pad_sequences(train_sequences, maxlen=max_length)
test_data = sequence.pad_sequences(test_sequences, maxlen=max_length)

In [ ]:
# Define the CNN model architecture
model = Sequential()
model.add(Embedding(10000, 100, input_length=max_length))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(64, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_data, train_df['seeds'], epochs=10, batch_size=32, validation_data=(test_data, test_df['seeds']))

In [ ]:
# Evaluate the model on the testing set
test_loss, test_acc = model.evaluate(test_data, test_df['label'])
print('Test accuracy:', test_acc)

# Music21


In [ ]:
import music21


In [ ]:
# Load a MIDI file
midi_file = music21.midi.MidiFile()
midi_file.open('/content/Chopin_Frédéric__François_-_Fantaisie_Improptu_Lilium100.mid')
midi_file.read()
midi_file.close()

# Convert the MIDI data to a music21 Stream object
stream = music21.midi.translate.midiTrackToStream(midi_file.tracks[0])


In [ ]:
# Analyze the key and mode
key = stream.analyze('key')
print("Key and mode: {} {}".format(key.tonic.name, key.mode))

Key and mode: C# major


In [ ]:
# Analyze the chords
chords = stream.chordify()
for chord in chords.recurse().getElementsByClass(music21.chord.Chord):
    print("Chord: {} {}".format(chord.root().name, chord.quality))

Chord: G# other
Chord: G# other
Chord: C# other
Chord: C# other
Chord: G# other
Chord: C# other
Chord: E other
Chord: C# other
Chord: G# other
Chord: C# other
Chord: G# other
Chord: C# other
Chord: E other
Chord: C# other
Chord: G# other
Chord: C# other
Chord: G# other
Chord: C# other
Chord: E other
Chord: C# other
Chord: G# other
Chord: C# other
Chord: G# other
Chord: C# other
Chord: E other
Chord: C# other
Chord: G# other
Chord: C# other
Chord: G# other
Chord: A major
Chord: E major
Chord: C# other
Chord: C# other
Chord: G# other
Chord: G# other
Chord: G# other
Chord: C# other
Chord: C# minor
Chord: E other
Chord: E other
Chord: E other
Chord: E- other
Chord: E- other
Chord: C# other
Chord: C# other
Chord: C other
Chord: C# other
Chord: C# minor
Chord: C# other
Chord: G# other
Chord: G# other
Chord: C# other
Chord: G# other
Chord: G# other
Chord: A major
Chord: A other
Chord: G# other
Chord: E other
Chord: E minor
Chord: C# minor
Chord: C# other
Chord: G# other
Chord: C# other
Chord:

In [ ]:
# Analyze the melodic patterns
for n in stream.flat.notes:
    if isinstance(n, music21.note.Note):
        print("Note: {} ({})".format(n.nameWithOctave, n.quarterLength))

Note: G#3 (7/3)
Note: G#2 (3.5)
Note: C#3 (2/3)
Note: C#2 (0.5)
Note: G#3 (0.5)
Note: C#4 (1/3)
Note: E4 (0.25)
Note: C#4 (0.25)
Note: G#3 (0.5)
Note: C#3 (1/3)
Note: G#3 (1/3)
Note: C#4 (1/3)
Note: E4 (0.25)
Note: C#4 (0.25)
Note: G#3 (0.5)
Note: C#3 (0.25)
Note: G#3 (0.25)
Note: C#4 (0.5)
Note: E4 (0.25)
Note: C#4 (0.25)
Note: G#3 (0.5)
Note: C#3 (1/3)
Note: G#3 (0.25)
Note: C#4 (0.5)
Note: E4 (0.25)
Note: C#4 (0.25)
Note: G#3 (0.5)
Note: C#3 (1/3)
Note: G#3 (0.25)
Note: C#4 (1/3)
Note: A4 (1/3)
Note: G#4 (0.25)
Note: E4 (0.25)
Note: G#4 (1/3)
Note: G#3 (0.5)
Note: C#5 (1/3)
Note: E3 (1/3)
Note: E5 (1/3)
Note: C#4 (0.25)
Note: C5 (0.25)
Note: C#5 (2/3)
Note: E4 (0.25)
Note: G#5 (0.25)
Note: G#3 (0.75)
Note: C#3 (1/3)
Note: G#3 (0.25)
Note: G#4 (0.25)
Note: A4 (1/3)
Note: C#4 (0.25)
Note: G#4 (0.25)
Note: E4 (1/3)
Note: G4 (0.25)
Note: C#4 (0.25)
Note: G#4 (1/3)
Note: C#5 (1/3)
Note: G#3 (1/3)
Note: E5 (1/3)
Note: E3 (1/3)
Note: E-5 (1/3)
Note: G#3 (1/3)
Note: C#5 (2/3)
Note: E4 (1/3)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Analyze the rhythm
for n in stream.flat.notes:
    if isinstance(n, music21.note.Note):
        print("Note: {} ({})".format(n.nameWithOctave, n.quarterLength))

Note: G#3 (7/3)
Note: G#2 (3.5)
Note: C#3 (2/3)
Note: C#2 (0.5)
Note: G#3 (0.5)
Note: C#4 (1/3)
Note: E4 (0.25)
Note: C#4 (0.25)
Note: G#3 (0.5)
Note: C#3 (1/3)
Note: G#3 (1/3)
Note: C#4 (1/3)
Note: E4 (0.25)
Note: C#4 (0.25)
Note: G#3 (0.5)
Note: C#3 (0.25)
Note: G#3 (0.25)
Note: C#4 (0.5)
Note: E4 (0.25)
Note: C#4 (0.25)
Note: G#3 (0.5)
Note: C#3 (1/3)
Note: G#3 (0.25)
Note: C#4 (0.5)
Note: E4 (0.25)
Note: C#4 (0.25)
Note: G#3 (0.5)
Note: C#3 (1/3)
Note: G#3 (0.25)
Note: C#4 (1/3)
Note: A4 (1/3)
Note: G#4 (0.25)
Note: E4 (0.25)
Note: G#4 (1/3)
Note: G#3 (0.5)
Note: C#5 (1/3)
Note: E3 (1/3)
Note: E5 (1/3)
Note: C#4 (0.25)
Note: C5 (0.25)
Note: C#5 (2/3)
Note: E4 (0.25)
Note: G#5 (0.25)
Note: G#3 (0.75)
Note: C#3 (1/3)
Note: G#3 (0.25)
Note: G#4 (0.25)
Note: A4 (1/3)
Note: C#4 (0.25)
Note: G#4 (0.25)
Note: E4 (1/3)
Note: G4 (0.25)
Note: C#4 (0.25)
Note: G#4 (1/3)
Note: C#5 (1/3)
Note: G#3 (1/3)
Note: E5 (1/3)
Note: E3 (1/3)
Note: E-5 (1/3)
Note: G#3 (1/3)
Note: C#5 (2/3)
Note: E4 (1/3)

# Tonal
Tonal is a Python library that provides tools for music analysis and composition. Tonal can be used to extract musical features from MIDI files, including key, mode, and chord progression, which can be used to infer the valence, arousal, and dominance of the music.

In [ ]:
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install mido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 6.4 MB/s eta 0:00:00


In [ ]:
import mido
import librosa

# Load the MIDI file
midi_file = mido.MidiFile('Chopin_Frédéric__François_-_Fantaisie_Improptu_Lilium100.mid')

# Extract the note and timing information from the MIDI file
notes = []
durations = []
for message in midi_file.play():
    if message.type == 'note_on':
        notes.append(message.note)
        durations.append(message.time)

# Compute the valence, arousal, and dominance from the note and timing information
spectral_centroid = librosa.feature.spectral_centroid(notes)
spectral_bandwidth = librosa.feature.spectral_bandwidth(notes)
valence = (spectral_centroid - spectral_bandwidth).mean()
arousal = spectral_centroid.mean()
dominance = (-spectral_centroid + spectral_bandwidth).mean()

# Print the results
print('Valence:', valence)
print('Arousal:', arousal)
print('Dominance:', dominance)

TypeError: ignored